# Enviroment Setup

In [11]:
!git clone https://github.com/Kolibri25/WINDIFY_AI


Cloning into 'WINDIFY_AI'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 111 (delta 39), reused 71 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 86.80 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [18]:
cd WINDIFY_AI

[Errno 2] No such file or directory: 'WINDIFY_AI'
/content/WINDIFY_AI/WINDIFY_AI


Setup environment

In [3]:
!pip install -r requirements.txt

In [13]:
import scripts.download_barra as dnb
import scripts.load_data as ld
from scripts.model_input import create_uv_tensor, create_uv_tensor_era5, split_tensor
import models.unet_model


In [14]:
import xarray as xr
import torch
import numpy as np
import matplotlib.pyplot as plt

Below purely to document BARRA data download and import procedure:

In [ ]:
'''

# download barrac2 data from months of Oct, Nov, Dec, Jan from 1979 to 2024
# lat/lon values and variables are hard-coded in the script
# need to check download path

dnb.main()

# download era5
# TODO: write script to download era5 data

'''

In [ ]:
'''
# combine files into single dataset per variable
# BARRA-C2
ld.load_and_save_barra_c2()
# ERA5
ld.load_and_save_era5()

Importing previously downloaded data using gdown from Google Drive folder

In [5]:
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1xwqJ8Ku_US5Q7MRvExKC26TqCUroQ6QE?usp=sharing

Retrieving folder contents
Retrieving folder 1_rHNYO880eYBNB0w7ciNWKxCop2oq4Na ERA5
Processing file 1biSKBWzXjYk8e7UWYl-QJs-bP4z0FWqe combined_era5.nc
Retrieving folder 1f2DGtE-oKAuXlnp2re9byp4UbMitIFOf BARRA-C2
Processing file 1MPIFRs3SoilpeLTMn7P21NyiRAOyZNIB uas_gesamt.nc
Processing file 1hkWS9IlTFCIS0BGv0Yl0WbgkILRsaPa_ vas_gesamt.nc
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1biSKBWzXjYk8e7UWYl-QJs-bP4z0FWqe
To: /content/WINDIFY_AI/metfut_data/ERA5/combined_era5.nc
100% 70.3M/70.3M [00:00<00:00, 91.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1MPIFRs3SoilpeLTMn7P21NyiRAOyZNIB
From (redirected): https://drive.google.com/uc?id=1MPIFRs3SoilpeLTMn7P21NyiRAOyZNIB&confirm=t&uuid=742e2f3d-f1d1-4267-88ef-16675a907a68
To: /content/WINDIFY_AI/metfut_data/BARRA-C2/uas_gesamt.nc
100% 1.72G/1.72G [00:21<00:00, 80.5MB/s]
Downloading...
From (original): https

In [6]:
!mv metfut_data/* /content/WINDIFY_AI/data/

In [16]:
# load target data (BARRA-C2)

uas_ds = xr.open_dataset('/content/WINDIFY_AI/data/BARRA-C2/uas_gesamt.nc')
vas_ds = xr.open_dataset('/content/WINDIFY_AI/data/BARRA-C2/vas_gesamt.nc')

# load input data (ERA5)

era5_ds = xr.open_dataset('/content/WINDIFY_AI/data/ERA5/combined_era5.nc')

# Data Preprocessing

Steps for Target ie. BARRA-C2:

1. Drop grid points to match model dimensions (divisible by 16).
2. Normalise the data.


Steps for Input ie. ERA5

1.   Drop grid points to match model dimensions (divisible by 16).
2.   Regrid EAR5 to a high-resolution grid of BARRA-C2
3.  Normalise the data.


Finally, we stack the input/target together as PyTorch tensors and split the data.

## Target: BARRA-C2

In [17]:
# crop to make dimensions divisible by 16

from scripts.crop_to_div16 import crop_to_multiple_of_16

uas_ds = crop_to_multiple_of_16(uas_ds, lat_name='lat', lon_name='lon')
vas_ds = crop_to_multiple_of_16(vas_ds, lat_name='lat', lon_name='lon')


ModuleNotFoundError: No module named 'scripts.crop_to_div16'

In [ ]:
# normalise the target data

from scripts.normalize_data import normalize_dataset

uas_ds, uas_mean, uas_std = normalize_data(uas_ds,"uas")
vas_ds, vas_mean, vas_std = normalize_data(vas_ds,"vas")

## Input: ERA5

In [ ]:
# regrid to barra grid

era5_interp = era5_ds.interp(
    lat=uas_ds.lat,
    lon=uas_ds.lon,
    method='nearest'
)

In [ ]:
# normalise input data

u10_ds, u10_mean, u10_std = normalize_data(era5_interp,"u10")["u10"]
v10_ds, v10_mean, v10_std = normalize_data(era5_interp,"v10")["v10"]

In [ ]:
# convert to PyTorch tensors

target = mi.create_uv_tensor(uas_ds,vas_ds)
input = mi.create_uv_tensor_era5(u10_ds,v10_ds)


In [ ]:
# check shapes

print(target.shape)
print(input.shape)

In [ ]:
# save in case of runtime shortage

from google.colab import drive
drive.mount('/content/drive')

output_path = "/content/drive/MyDrive/metfut_data/processed_data/"

import os

# Ensure trailing slash and directory exists
os.makedirs(output_path, exist_ok=True)

# Use os.path.join to avoid hardcoding slashes
torch.save(target, os.path.join(output_path, 'target.pt'))
torch.save(input, os.path.join(output_path, 'input.pt'))



In [ ]:
# train/validation/test split

x_train, x_val, x_test = split_tensor(input)
y_train, y_val, y_test = split_tensor(target)